In [1]:
import conllu
import os

In [26]:
narc_path = "../output/aligned/no-narc_bokmaal/narc_bokmaal_dev.conllu"
data = conllu.parse(open(narc_path, "r").read())

In [27]:
# a new document will always start with "newdoc id" in the metadata field
def is_new_doc(sample):
    return "newdoc id" in sample.metadata
# group all documents by their id.
from collections import defaultdict
grouped_docs = defaultdict(list)
current_doc = None
for sent in data:
    if is_new_doc(sent):
        current_doc = sent.metadata["newdoc id"]
    grouped_docs[current_doc].append(sent)

In [28]:
grouped_docs.keys()

dict_keys(['ap~20050210-762251', 'ap~20081210-1411542', 'ap~20081210-1546270', 'ap~20081210-1564010', 'ap~20081210-1775472', 'ap~20090401-3010501', 'ap~20090803-3199497', 'ap~20090805-3202217', 'ap~20090825-3233467', 'ap~20090905-3252356', 'ap~20090911-3262518', 'ap~20091022-3333021', 'bt~BT-20120916-2765289a', 'db~20081117-3745306', 'db~20081118-3754590', 'db~20081118-3758669', 'db~20081118-3759012', 'db~20081128-3858534a', 'db~20081202-3901555', 'db~20081206-3954583', 'kk~20110826-59215', 'kk~20110827-59218', 'spbm~20050822-508220301', 'spbm~20050822-508220303', 'spbm~20050822-508220304', 'spbm~20050822-508220308', 'spbm~20050822-508220309', 'spbm~20050822-508220311', 'spbm~20050822-508220313', 'vg~VG-20121202-10056280', 'vg~VG-20121211-10071599'])

In [29]:
sample = grouped_docs["ap~20050210-762251"]
sample[:3]

[TokenList<Pinefull, død, for, den, irske, pub, ?, metadata={newdoc id: "ap~20050210-762251", global.Entity: "eid-etype-head-other", sent_id: "016056", text: "Pinefull død for den irske pub?"}>,
 TokenList<Irene, diskuterer, over, mørkebrunt, øl, ., metadata={newpar: "None", sent_id: "016057", text: "Irene diskuterer over mørkebrunt øl."}>,
 TokenList<Micheál, Martin, er, på, alles, lepper, ., metadata={sent_id: "016058", text: "Micheál Martin er på alles lepper."}>]

In [ ]:
# Step 1: Create a dictionary to store the reference IDs and their corresponding spans.
references = {}


In [75]:
import re

CONLL_MENTION_PATTERN = re.compile(
    r'(?:\((?P<mono>\d+)\)|\((?P<start>\d+)|(?P<end>\d+)\))')

def compute_mentions(columns):
    pending = defaultdict(list)
    mentions = []
    i = 0

    for i, col in enumerate(columns):
        for m in CONLL_MENTION_PATTERN.finditer(col):
            if m.lastgroup == 'mono':
                pos = (i, i+1)
                chain = int(m.group(m.lastgroup))
                mentions.append((pos, chain))
            elif m.lastgroup == 'start':
                chain = int(m.group(m.lastgroup))
                if not chain in pending:
                    pending[chain] = []
                pending[chain].append(i)
            elif m.lastgroup == 'end':
                chain = int(m.group(m.lastgroup))
                pos = (pending[chain].pop(), i+1)
                mentions.append((pos, chain))
            else:
                assert False

    return mentions

def compute_chains(columns):
    chains = dict()
    for (start, stop), chain_id in compute_mentions(columns):
        end = stop - 1
        if chain_id not in chains:
            chains[chain_id] = []
        chains[chain_id].append((start, end))
    return list(chains.values())

columns = ["(1", "*", "1)(2)(3", "*", "*", "3)", "*", "*", "(1", "1)"]
for i, c in enumerate(columns):
    print(i, c)

compute_chains(columns)


0 (1
1 *
2 1)(2)(3
3 *
4 *
5 3)
6 *
7 *
8 (1
9 1)


[[(0, 2), (8, 9)], [(2, 2)], [(2, 5)]]

In [ ]:
def build_references(ents):
    # from a list of entities, ["1)", "2)"], build a dictionary of references
    references = {}


    return []

In [41]:
from collections import defaultdict
references = defaultdict(list)
# parse a document:
token_idx = 0
for s_id, sent in enumerate(sample):
    for token in sent:
        misc = token.get("misc", {})
        ents = misc.get("Entity", None)
        if ents:
            ents = ents.split("§")
        print(token, token_idx, ents)
        token_idx += 1
    if s_id > 2:
        break
    

Pinefull 0 ['(1']
død 1 None
for 2 None
den 3 ['(2']
irske 4 None
pub 5 ['2)', '1)']
? 6 None
Irene 7 ['(3)']
diskuterer 8 None
over 9 None
mørkebrunt 10 ['(4']
øl 11 ['4)']
. 12 None
Micheál 13 ['(5373']
Martin 14 ['5373)']
er 15 None
på 16 None
alles 17 ['(6']
lepper 18 ['6)']
. 19 None
Helseministeren 20 ['(5373)']
innfører 21 None
totalt 22 ['(162447']
røykeforbud 23 ['162447)']
på 24 None
pubene 25 ['(4514)']
i 26 None
morgen 27 ['(10)']
. 28 None


In [31]:
from collections import defaultdict

entities = defaultdict(list)
# this is a list of lists, each being a mention tuple (start, end, entity_id)


from collections import defaultdict
grouped_docs = defaultdict(list)
current_doc = None
for sent in data:
    if is_new_doc(sent):
        current_doc = sent.metadata["newdoc id"]
    grouped_docs[current_doc].append(sent)